In [ ]:
# 숙제
# iris.csv 데이터를 로딩한 다음
# 분류망을 구성하세요
# parameter tuning을 구현하세요( pipeline도 함께 )

### 데이터 loading

In [5]:
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

In [6]:
dataset = pd.read_csv('iris.csv', header=None)
dataset.head()
dataset = dataset.iloc[1:,:]
print(dataset.shape)
dataset.head()


(150, 5)


,0,1,2,3,4
1,6.4,2.8,5.6,2.2,2
2,5.0,2.3,3.3,1,1
3,4.9,2.5,4.5,1.7,2
4,4.9,3.1,1.5,0.1,0
5,5.7,3.8,1.7,0.3,0


In [7]:
X  = dataset.iloc[:,:4]
Y = dataset.iloc[:,4]
Y = np.asarray(Y)
print(X.shape)
print(Y.shape)

(150, 4)
(150,)


### 기본 실행문
- gridsearchcv 사용

In [10]:
def baseline_model() :
    model = Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(3, activation = 'relu'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model


In [11]:
model = KerasClassifier(build_fn=baseline_model, verbose=1 )
batch_size = [10,20,30]
epochs = [10,50,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)
print("최적스코어 : {:.4f}  사용한 파라미터 조합 : {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("{:.2f} ({:.2f})    with {}".format(mean,std,param))

Epoch 1/10
150/150 [==============================] - 0s 2ms/step - loss: 10.7454 - accuracy: 0.3333
Epoch 2/10
150/150 [==============================] - 0s 53us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 3/10
150/150 [==============================] - 0s 60us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 4/10
150/150 [==============================] - 0s 53us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 5/10
150/150 [==============================] - 0s 100us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 6/10
150/150 [==============================] - 0s 53us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 7/10
150/150 [==============================] - 0s 60us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 8/10
150/150 [==============================] - 0s 53us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 9/10
150/150 [==============================] - 0s 53us/step - loss: 10.7454 - accuracy: 0.3333
Epoch 10/10
150/150 [==============================] - 0s 60us/step - loss: 10.745

- ++ 파이프라인

In [61]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
estimators = []
estimators.append(('MinMax', MinMaxScaler()))
estimators.append(('classifier', KerasClassifier(build_fn=baseline_model, verbose=1 )))
pipeline = Pipeline(estimators)
param_grid = {'classifier__batch_size':batch_size, 'classifier__epochs':epochs}

grid = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)
print("최적스코어 : {:.4f}  사용한 파라미터 조합 : {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("{:.2f} ({:.2f})    with {}".format(mean,std,param))


Epoch 1/10
150/150 [==============================] - 2s 13ms/step - loss: 6.0096 - acc: 0.3333
Epoch 2/10
150/150 [==============================] - 0s 193us/step - loss: 5.8687 - acc: 0.3400
Epoch 3/10
150/150 [==============================] - 0s 145us/step - loss: 5.7554 - acc: 0.3600
Epoch 4/10
150/150 [==============================] - 0s 133us/step - loss: 5.6217 - acc: 0.5600
Epoch 5/10
150/150 [==============================] - 0s 193us/step - loss: 5.4878 - acc: 0.6533
Epoch 6/10
150/150 [==============================] - 0s 153us/step - loss: 5.4168 - acc: 0.6667
Epoch 7/10
150/150 [==============================] - 0s 140us/step - loss: 5.3786 - acc: 0.6667
Epoch 8/10
150/150 [==============================] - 0s 133us/step - loss: 5.3727 - acc: 0.6667
Epoch 9/10
150/150 [==============================] - 0s 133us/step - loss: 5.3727 - acc: 0.6667
Epoch 10/10
150/150 [==============================] - 0s 100us/step - loss: nan - acc: 0.6267
최적스코어 : 0.4467  사용한 파라미터 조합 : {'c

## 다양한 모델 적용

- 모델 설정

In [62]:
def baseline_model(init_mode = 'normal', optimizer = 'relu') :
    model = Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer=init_mode, activation=optimizer))
    model.add(Dense(3, activation = optimizer))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

def larger_model(init_mode = 'normal', optimizer = 'relu'):
    model = Sequential()
    model.add(Dense(13, input_dim=4, kernel_initializer=init_mode, activation=optimizer))
    model.add(Dense(6, kernel_initializer=init_mode, activation=optimizer))
    model.add(Dense(3, kernel_initializer=init_mode, activation=optimizer))
    model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])
    return model

def wider_model(init_mode = 'normal', optimizer = 'relu'):
    model = Sequential()
    model.add(Dense(20, input_dim=4, kernel_initializer=init_mode, activation=optimizer))
    model.add(Dense(3, kernel_initializer=init_mode, activation=optimizer))
    model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])
    return model



- 파라미터 설정

In [66]:
init_mode = ['uniform', 'lecun_uniform','normal','glorot_normal','glorot_uniform','he_normal','he_uniform'] 
optimizer = ['softmax','relu', 'softplus', 'softsign','tanh','sigmoid']
batch_size = [10,20,30]
epochs = [10,50,100]


- baseline_model()

In [67]:
model_list = [baseline_model(), larger_model(),wider_model()]

In [ ]:
estimators = []
estimators.append(('MinMax', MinMaxScaler()))
estimators.append(('classifier', KerasClassifier(build_fn=baseline_model, verbose=1 )))
pipeline = Pipeline(estimators)
param_grid = {'classifier__batch_size':batch_size,'classifier__epochs':epochs,
         "classifier__init_mode":init_mode,'classifier__optimizer':optimizer}

grid = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)
print("최적스코어 : {:.4f}  사용한 파라미터 조합 : {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("{:.2f} ({:.2f})    with {}".format(mean,std,param))

- larger_model()

In [ ]:
estimators = []
estimators.append(('MinMax', MinMaxScaler()))
estimators.append(('classifier', KerasClassifier(build_fn=larger_model, verbose=1 )))
pipeline = Pipeline(estimators)
param_grid = {'classifier__batch_size':batch_size,'classifier__epochs':epochs,
         "classifier__init_mode":init_mode,'classifier__optimizer':optimizer}

grid = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)
print("최적스코어 : {:.4f}  사용한 파라미터 조합 : {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("{:.2f} ({:.2f})    with {}".format(mean,std,param))

- wider_model()

In [ ]:
estimators = []
estimators.append(('MinMax', MinMaxScaler()))
estimators.append(('classifier', KerasClassifier(build_fn=wider_model, verbose=1 )))
pipeline = Pipeline(estimators)
param_grid = {'classifier__batch_size':batch_size,'classifier__epochs':epochs,
         "classifier__init_mode":init_mode,'classifier__optimizer':optimizer}

grid = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X,Y)
print("최적스코어 : {:.4f}  사용한 파라미터 조합 : {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("{:.2f} ({:.2f})    with {}".format(mean,std,param))